In [1]:
#!pip install folium
#!pip install pyproj

In [2]:
import pandas as pd
import folium
import math

from itertools import combinations
from pyproj import Proj, transform
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

### 데이터 다운 및 전처리

In [3]:
Gyeongju = pd.read_excel('경주시예상좌표.xlsx')
Gyeongju = Gyeongju[['대구분명칭', '위도', '경도']]
Gyeongju.columns = ['이름', '위도', '경도']
Gyeongju.head()

,이름,위도,경도
0,북성로 노상주차장,35.847224,129.215349
1,시외버스터미널 노상주차장,35.839648,129.203010
2,시청 노상주차장,35.856266,129.225735
3,시외버스터미널 노상주차장,35.840799,129.203396
4,동문로 노상주차장,35.845753,129.214507


In [4]:
market = pd.read_excel('경주_편의점.xlsx')
market.head()

,이름,위도,경도
0,오션힐즈편의점,35.664234,129.405007
1,CU 경주석계공단점,35.664296,129.305564
2,이마트24 경주양남비치점,35.666837,129.459148
3,이마트24 경주석계관문점,35.669607,129.301811
4,이마트24 경주문산공단점,35.671310,129.323321


### 위,경도 좌표변환

In [5]:
proj_UTMK = Proj(init='epsg:5178')
proj_WGS84 = Proj(init = 'epsg:4326')

In [6]:
for i in  tqdm(range(len(Gyeongju))) :
    x1, y1 = transform(proj_WGS84, proj_UTMK, Gyeongju['경도'][i], Gyeongju['위도'][i])
    Gyeongju['경도'][i], Gyeongju['위도'][i] = x1, y1

100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [00:04<00:00, 11.18it/s]


In [7]:
market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   이름      270 non-null    object 
 1   위도      270 non-null    float64
 2   경도      270 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.5+ KB


In [8]:
for i in  tqdm(range(len(market))) :
    x1, y1 = transform(proj_WGS84, proj_UTMK, market['경도'][i], market['위도'][i])
    market['경도'][i], market['위도'][i] = x1, y1

100%|████████████████████████████████████████████████████████████████████████████████| 270/270 [00:23<00:00, 11.62it/s]


### F와 L사이의 최단거리 행열 만들기

$F_i$ - Gyeongju, 경주시 공영시설 좌표, m = 54

$L_j$ - market, 경주시 편의점 좌표, n = 270 

$c_{ij}$ - $F_i$와 $L_j$의 최단거리

In [9]:
F_list = []
L_list = []
for i in range(len(Gyeongju)) :
    name = 'F_{}'.format(i)
    F_list.append(name)
for i in range(len(market)) :
    name = 'L_{}'.format(i)
    L_list.append(name)

In [10]:
length_df = pd.DataFrame(columns = F_list, index = L_list)
for i in range(len(length_df)) :
    for j, col in enumerate(length_df.columns) :
        square_sum = ((Gyeongju['위도'][j] - market['위도'][i]) ** 2) + ((Gyeongju['경도'][j] - market['경도'][i]) ** 2)
        distance = math.sqrt(square_sum)
        length_df[col][i] = distance

###  P-Median

In [11]:
char_list = []
for p in range(3, 11) :
    col_sum = list(length_df.sum(axis = 0))
    col_sum_sort = sorted(col_sum)
    min_value = col_sum_sort[:3]
    
    min_index = []
    for value in min_value :
        min_index.append(col_sum.index(value))
    
    col_sum_tuple = []
    for i in range(len(col_sum)) :
        tup = (i, col_sum[i])
        col_sum_tuple.append(tup)
    
    col_sum_tuple.sort(key = lambda x: x[1])
    col_sum_tuple = col_sum_tuple[:2*p]
    
    p_list_set = [col_sum_tuple[j:j+p] for j in range(p)]
    
    min_sum_list = []
    for k in range(len(p_list_set)) :
        tup_check = []
        for tt in p_list_set[k] :
            tup_check.append(f'F_{tt[0]}')
        check_df = length_df[tup_check]
        check_df['min'] = 0
        for l in range(len(check_df)) :
            check_df['min'][l] = min(check_df.iloc[l][:-1])
        min_sum_value = sum(check_df['min'])
        min_sum_list.append(min_sum_value)
    
    final_index = min_sum_list.index(min(min_sum_list))
    final_set = p_list_set[final_index]
    
    final_set.sort(key = lambda x : x[0])
    
    final_idx = [tp[0] for tp in final_set]
    final_Gyeongju = Gyeongju.iloc[final_idx, :]
    
    final_Gyeongju.reset_index(drop = True, inplace = True)
    char_list.append(char for char in final_Gyeongju['이름'])

In [12]:
final_kk = []
for ch in char_list :
    final_kk += ch

In [13]:
pd.Series(final_kk).value_counts()

제2공영주차장         9
북정로 노상주차장       8
북성로 노상주차장       6
팔우정 노상주차장       6
동문로 노상주차장       5
중앙로 노상주차장       4
중심상가주차장         4
태종로 노상주차장       3
시청사주차장          3
성동시장공영주차장       2
시청 노상주차장        1
경주문화원앞 노상주차장    1
dtype: int64

In [28]:
check_index = pd.Series(final_kk).value_counts().index[:4]

In [29]:
Gyeongju_index = []
for check in check_index :
    for i in range(len(Gyeongju)) :
        if Gyeongju['이름'][i] == check :
            Gyeongju_index.append(i)
            break

In [30]:
Gyeongju_index

[27, 17, 0, 12]

In [31]:
Gyeongju_sub = Gyeongju.iloc[Gyeongju_index, :]
Gyeongju_sub.reset_index(drop = True, inplace = True)
Gyeongju_sub

,이름,위도,경도
0,제2공영주차장,1.761678e+06,1.155234e+06
1,북정로 노상주차장,1.761803e+06,1.155082e+06
2,북성로 노상주차장,1.762242e+06,1.155099e+06
3,팔우정 노상주차장,1.761793e+06,1.155190e+06


### 시장데이터 및 최종공영시설 좌표 위경도 변환

In [32]:
Gyeongju_sijang = pd.read_excel('경주시장.xlsx')
Gyeongju_sijang.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [33]:
Gyeongju_sijang = Gyeongju_sijang[['시장명', '위도', '경도']]

In [34]:
Gyeongju_sijang

,시장명,위도,경도
0,건천시장,1762224.781,1145157.267
1,외동시장,1747751.553,1165383.951
2,양남시장,1743601.330,1177911.095
3,감포시장,1757951.668,1181086.004
4,산내시장,1752070.531,1140006.141
5,불국시장,1754453.038,1162998.331
6,불국사상가시장,1755659.466,1165345.034


In [35]:
for i in  tqdm(range(len(Gyeongju_sijang))) :
    x1, y1 = transform(proj_UTMK, proj_WGS84, Gyeongju_sijang['경도'][i], Gyeongju_sijang['위도'][i])
    Gyeongju_sijang['경도'][i], Gyeongju_sijang['위도'][i] = x1, y1

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 10.57it/s]


In [43]:
for i in  tqdm(range(len(Gyeongju_sub))) :
    x1, y1 = transform(proj_UTMK, proj_WGS84, Gyeongju_sub['경도'][i], Gyeongju_sub['위도'][i])
    Gyeongju_sub['경도'][i], Gyeongju_sub['위도'][i] = x1, y1

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11.08it/s]


In [44]:
m = folium.Map(location = [35.856371579014926, 129.2247815402173], zoom_start = 10)

In [45]:
#m.save('map.html')
# m

In [46]:
for i in range(len(Gyeongju_sijang)) :
    long = Gyeongju_sijang['위도'][i]
    lat = Gyeongju_sijang['경도'][i]
    name = Gyeongju_sijang['시장명'][i]
    folium.Marker(location = [long, lat],
                  popup = name,
                 icon = folium.Icon(color = 'yellow',
                                    icon = 'star',
                                    size = 0.1)).add_to(m)

In [47]:
for i in range(len(Gyeongju_sub)) :
    long = Gyeongju_sub['위도'][i]
    lat = Gyeongju_sub['경도'][i]
    name = Gyeongju_sub['이름'][i]
    folium.Marker(location = [long, lat],
                  popup = name,
                 icon = folium.Icon(color = 'blue',
                                    icon = 'flag',
                                    size = 0.1)).add_to(m)

In [48]:
m

위기 시장들간의 거리는 멀지만 주변시설이 많다는 점과 경주시 중심에 위치하여 이용자가 많을것으로 고려되어 해당지역에 전통시장 드라이브스루를 설치하게 되면 이용률이 증가할 것이다. 또한 해당지역에 드라이브스루를 설치함과 동시에 위기시장에서 물품을 납품받아 시설을 운영하면 상생효과를 볼 수 있을 것이다.